### Run analysis to probe for the tagged frequencies
`sbatch --time=04:00:00 --cpus-per-task=1 --mem-per-cpu=8GB --account=def-mmur {script.sh}` # basic
`sbatch --time=11:00:00 --cpus-per-task=1 --mem-per-cpu=12GB --account=def-mmur {script.sh}` # IMall

In [1]:
from pathlib import Path
import itertools
import glob
import os
from copy import deepcopy

from ComputeCanada.frequency_tagging.study_info import (
    frequency_tagging_settings,
    attention_settings,
    setting_exceptions,
)

"""
Functions
"""

def search(base_dir, wildcard, error=True):
    search_path = Path(base_dir) / wildcard
    files = glob.glob(str(search_path))

    if not files:
        if error:
            raise FileNotFoundError(f"No files were found in: {search_path}")
        else:
            return []

    return files

def get_task_id_bases(relevant_task_ids, experiment_search_frequencies, mri_id, desc_id):

    if desc_id == "basic":
        _relevant_task_ids = []
        for task_id in experiment_search_frequencies.keys():
            task_id_base = task_id.split("_")[0]
            if task_id_base in relevant_task_ids:
                _relevant_task_ids.append(task_id_base)
    
    if desc_id == "IMall":
        _relevant_task_ids = []
        for task_id, search_fs in experiment_search_frequencies.items():
            task_id_base = task_id.split("_")[0]
            if len(search_fs) == 2 and task_id_base in relevant_task_ids:
                _relevant_task_ids.append(task_id_base)
                search_fs.sort()
                # Second order frequencies
                experiment_search_frequencies[task_id].append(round(search_fs[1] - search_fs[0], 3))
                experiment_search_frequencies[task_id].append(round(search_fs[1] + search_fs[0], 3))
                experiment_search_frequencies[task_id].append(round(search_fs[0]*2, 3))
                experiment_search_frequencies[task_id].append(round(search_fs[1]*2, 3))
                # Third order frequencies
                experiment_search_frequencies[task_id].append(round(search_fs[0]*2-search_fs[1], 3))
                experiment_search_frequencies[task_id].append(round(search_fs[1]*2-search_fs[0], 3))

    _relevant_task_ids = list(set(_relevant_task_ids))

    return _relevant_task_ids, experiment_search_frequencies

def get_subject_ids(experiment_id, mri_id):
    # Read to find subject ids
    _dir = f"/data/{experiment_id}/{mri_id}/bids"
    sub_ids = [Path(i).stem.split('sub-')[1] for i in search(_dir, 'sub-*')]
    sub_ids.sort()

    return sub_ids

In [2]:
main_scripts_dir = Path("/data/scripts/03b_run_level_bootstrap")

SMOOTH_OVERRIDE = 0
SETTINGS = deepcopy(attention_settings)
experiment_ids, mri_ids, oscprep_dir, smooth_mm, relevant_task_ids, experiment_search_frequencies, TR, stim_start_time, stim_rampup_time, experiment_stim_end, CONTAINER, image_type = SETTINGS.values()
if SMOOTH_OVERRIDE is not None:
    smooth_mm = SMOOTH_OVERRIDE
denoising_strategy = "00_experiment-min+motion24+wmcsfmean"
n_batches = 1
n_iterations = 5
desc_ids = ["IMall"]
pval_ids = ["uncp", "fdrp"]


for pval_id, desc_id in itertools.product(pval_ids, desc_ids):

    assert pval_id in ['fdrp', 'uncp'], f"{pval_id} must be set to fdrp or uncp."

    # Output sbatch scripts here
    scripts_dir = main_scripts_dir / desc_id
    scripts_dir = Path(scripts_dir)
    if not scripts_dir.exists():
        scripts_dir.mkdir(parents=True)

    for experiment_id, mri_id in itertools.product(experiment_ids, mri_ids):
        sub_ids = get_subject_ids(experiment_id, mri_id)
        for sub_id in sub_ids:
            
            # change settings based on exceptions
            settings = setting_exceptions(experiment_id, mri_id, f"sub-{sub_id}", deepcopy(SETTINGS))
            _, _, oscprep_dir, smooth_mm, relevant_task_ids, experiment_search_frequencies, TR, stim_start_time, stim_rampup_time, experiment_stim_end, CONTAINER, image_type = settings.values()
            if SMOOTH_OVERRIDE is not None:
                smooth_mm = SMOOTH_OVERRIDE
            task_id_bases, experiment_search_frequencies = get_task_id_bases(relevant_task_ids, experiment_search_frequencies, mri_id, desc_id)
            time_window = (
                stim_start_time + stim_rampup_time, 
                experiment_stim_end[f"{relevant_task_ids[0]}_{mri_id}"]
            )

            for task_id_base, batch_id in itertools.product(task_id_bases, range(n_batches)):

                base_out_dir = f"/scratch/fastfmri/experiment-{experiment_id}_mri-{mri_id}_smooth-{smooth_mm}_truncate-{time_window[0]}-{time_window[1]}_n-{n_iterations}_batch-{str(batch_id).zfill(2)}_desc-{desc_id}_pval-{pval_id}_bootstrap"

                _TR = TR[f"{task_id_base}_{mri_id}"]
                search_frequencies = experiment_search_frequencies[f"{task_id_base}_{mri_id}"]

                #print(experiment_id, mri_id, sub_id, task_id_base, batch_id, smooth_mm)

                try:
                    # If it does not detect anything according to search(), then output a batch script
                    fs = search(f"{base_out_dir}/sub-{sub_id}/bootstrap", f"*{task_id_base}*")
                except:
                    # Verbose, print some stuff
                    print(
                        experiment_id, 
                        mri_id, 
                        sub_id,
                        task_id_base, 
                        smooth_mm, 
                        search_frequencies, 
                        time_window, 
                        _TR,
                        denoising_strategy,
                        n_iterations,
                        base_out_dir,
                    )
                    # Print sbatch scripts to `scripts_dir`
                    if pval_id == "fdrp":
                        txt = f"""#!/bin/bash
module load apptainer/1.2.4

singularity run \\
--bind /project/def-mmur/gngo4/data/fastfmri:/data \\
--bind /project/def-tschmitz-ab/gngo4:/scratch \\
--bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
{CONTAINER} \\
python3 /opt/app/scripts/run_level_bootstrap.py \\
--experiment-id {experiment_id} \\
--mri-id {mri_id} \\
--sub-id {sub_id} \\
--task-id-base {task_id_base} \\
--smooth-mm {smooth_mm} \\
--search-frequencies {' --search-frequencies '.join([str(i) for i in search_frequencies])} \\
--time-window {' '.join([str(i) for i in time_window])} \\
--tr {_TR} \\
--denoising-strategy {denoising_strategy} \\
--n-iterations {n_iterations} \\
--base-out-dir {base_out_dir} \\
--fdr-correction
                        """
                    elif pval_id == "uncp":
                        txt = f"""#!/bin/bash
module load apptainer/1.2.4

singularity run \\
--bind /project/def-mmur/gngo4/data/fastfmri:/data \\
--bind /project/def-tschmitz-ab/gngo4:/scratch \\
--bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
{CONTAINER} \\
python3 /opt/app/scripts/run_level_bootstrap.py \\
--experiment-id {experiment_id} \\
--mri-id {mri_id} \\
--sub-id {sub_id} \\
--task-id-base {task_id_base} \\
--smooth-mm {smooth_mm} \\
--search-frequencies {' --search-frequencies '.join([str(i) for i in search_frequencies])} \\
--time-window {' '.join([str(i) for i in time_window])} \\
--tr {_TR} \\
--denoising-strategy {denoising_strategy} \\
--n-iterations {n_iterations} \\
--base-out-dir {base_out_dir}
                        """
                    cmd_path = f"{scripts_dir}/experiment-{experiment_id}_mri-{mri_id}_sub-{sub_id}_task-{task_id_base}_smooth-{smooth_mm}_truncate-{time_window[0]}-{time_window[1]}_n-{n_iterations}_batch-{str(batch_id).zfill(2)}_desc-{desc_id}_pval-{pval_id}_bootstrap.sh"

                    print(cmd_path)
                    with open(cmd_path, 'w') as f:
                        f.write(txt)

1_attention 7T 010 AttendInF1F2 0 [0.125, 0.2, 0.075, 0.325, 0.25, 0.4, 0.05, 0.275] (39, 219) 0.25 00_experiment-min+motion24+wmcsfmean 5 /scratch/fastfmri/experiment-1_attention_mri-7T_smooth-0_truncate-39-219_n-5_batch-00_desc-IMall_pval-uncp_bootstrap
/data/scripts/03b_run_level_bootstrap/IMall/experiment-1_attention_mri-7T_sub-010_task-AttendInF1F2_smooth-0_truncate-39-219_n-5_batch-00_desc-IMall_pval-uncp_bootstrap.sh
1_attention 7T 010 AttendInF1 0 [0.125, 0.2, 0.075, 0.325, 0.25, 0.4, 0.05, 0.275] (39, 219) 0.25 00_experiment-min+motion24+wmcsfmean 5 /scratch/fastfmri/experiment-1_attention_mri-7T_smooth-0_truncate-39-219_n-5_batch-00_desc-IMall_pval-uncp_bootstrap
/data/scripts/03b_run_level_bootstrap/IMall/experiment-1_attention_mri-7T_sub-010_task-AttendInF1_smooth-0_truncate-39-219_n-5_batch-00_desc-IMall_pval-uncp_bootstrap.sh
1_attention 7T 010 AttendAway 0 [0.125, 0.2, 0.075, 0.325, 0.25, 0.4, 0.05, 0.275] (39, 219) 0.25 00_experiment-min+motion24+wmcsfmean 5 /scratch/fa